In [3]:
from IPython import display
import math
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [4]:
import praw    #Python Reddit API Wrapper

In [5]:
reddit = praw.Reddit(client_id='hKlt61fjB5Unfw',
                     client_secret='YmPGZDvCrwt5RvmqID6mmfSSO-w',
                     user_agent='prakherg')

In [6]:
print(reddit)

In [7]:
headlines = {"created" : [],
             "score" : [] ,
             "title" : []}

In [8]:
for submission in reddit.subreddit('cryptocurrency').new(limit=None):
    headlines["created"].append(submission.created)
    headlines["score"].append(submission.score)
    headlines["title"].append(submission.title)
    display.clear_output()
    print(len(headlines))

3


In [22]:
headlines_df = pd.DataFrame(headlines)
headlines_df = headlines_df.sort_values(by=["score"],ascending=False)

print(headlines_df["title"])

695    My uncle who told me Bitcoin is a scam 6 month...
195                     Weak hands always gonna complain
494    Venezuelan Buys 300 Kilos of Food with Just 61...
240                                       Gangnam, Seoul
67     Russian Agents Who Hacked DNC Emails Thought B...
680    Building of the National Bank of Slovakia last...
167    Manipulated ETH network gas prices look connec...
409    US Congress Calls Digital Currencies 'The Futu...
190    Bitcoin Cash Network Highly Centralized, 49% o...
453    Bitfinex is airdropping every EOS shitcoin, pr...
711    If cryptocurrency becomes commonplace and prov...
313    In response to Coinbase listing announcement, ...
333    Swiss stock exchange to integrates crypto curr...
315                                            Ok, Weird
320                                      Wage Cuckin' It
392      Apparently Nano isn't the fastest coin anymore.
94     Andreas Antonopoulos: When would crypto be suc...
584    1979 Computer Store Mana

In [23]:
import datetime as dt
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [24]:
_timestamp = headlines_df["created"].apply(get_date)

In [25]:
headlines_df = headlines_df.assign( timestamp = _timestamp) 

In [26]:
headlines_df.tail(5)

,created,score,title,timestamp
246,1.531590e+09,0,Cryptocurrency Market Update: Five Spikes for ...,2018-07-14 12:37:17
243,1.531590e+09,0,Ex-Macau gangster raises US$750 million in cry...,2018-07-14 12:46:23
242,1.531591e+09,0,Distributed Credit Chain [DCC] is a blockchain...,2018-07-14 12:54:57
241,1.531591e+09,0,Good time to accumulate your altcoin portfolio...,2018-07-14 13:00:30
358,1.531546e+09,0,Market Update July 13th - Vitalik Buterin Hope...,2018-07-14 00:24:03


In [27]:
headlines_df.to_csv('NewsArticles.csv',sep='\t')

# Labeling data

In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
results = []

for line in headlines_df["title"]:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

In [35]:
df = pd.DataFrame.from_records(results)
df = df[df["neu"] < 0.5].sort_values(by = ["pos","neg"],ascending = False)
df.head()

,compound,headline,neg,neu,pos
555,0.4767,Cryptocurrency with friends,0.000,0.392,0.608
248,0.4753,Bitcoin is powerful!,0.000,0.393,0.607
516,0.3818,Distributed Credit Chain,0.000,0.435,0.565
580,0.3818,Distributed Credit Chain,0.000,0.435,0.565
13,0.1280,"Ok, Weird",0.436,0.000,0.564


In [36]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,compound,headline,neg,neu,pos,label
555,0.4767,Cryptocurrency with friends,0.000,0.392,0.608,1
248,0.4753,Bitcoin is powerful!,0.000,0.393,0.607,1
516,0.3818,Distributed Credit Chain,0.000,0.435,0.565,1
580,0.3818,Distributed Credit Chain,0.000,0.435,0.565,1
13,0.1280,"Ok, Weird",0.436,0.000,0.564,0


In [37]:
headlines1_df = pd.merge(headlines_df, df,  left_on='title', right_on='headline')


In [38]:
headline1_df_filter = headlines1_df[headlines1_df["label"]!=0]

In [39]:
headline1_df_filter.drop_duplicates('title',keep = 'last').tail(5)

,created,score,title,timestamp,compound,headline,neg,neu,pos,label
23,1.531616e+09,0,Cryptocurrency with friends,2018-07-14 19:55:04,0.4767,Cryptocurrency with friends,0.000,0.392,0.608,1
24,1.531623e+09,0,Diversify your Crypto Assets into Precious Metals,2018-07-14 21:52:20,0.6597,Diversify your Crypto Assets into Precious Metals,0.000,0.481,0.519,1
25,1.531502e+09,0,Robinhood's Plan to Win the Crypto Exchange Wa...,2018-07-13 12:19:30,-0.7003,Robinhood's Plan to Win the Crypto Exchange Wa...,0.422,0.392,0.186,-1
26,1.531546e+09,0,Help save the good coin with the bad name!,2018-07-14 00:34:50,0.6800,Help save the good coin with the bad name!,0.199,0.284,0.517,1
27,1.531605e+09,0,"HALP, ENTIRE TBDice NODE STOPPED BOOPIN', MUCH...",2018-07-14 16:43:51,-0.7404,"HALP, ENTIRE TBDice NODE STOPPED BOOPIN', MUCH...",0.511,0.489,0.000,-1


In [122]:
headline1_df_filter['timestamp'] = pd.to_datetime(headline1_df_filter.timestamp) 

C:\Users\gupta\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [199]:
headline1_df_filter.head(6)

,created,score,title,timestamp,compound,headline,neg,neu,pos,label
1,1.525237e+09,1,UnikoinGold the First Crypto Officially Approv...,2018-05-01 23:57:51,0.4215,UnikoinGold the First Crypto Officially Approv...,-0.000,0.833,0.167,1
2,1.525237e+09,1,UnikoinGold the First Crypto Officially Approv...,2018-05-01 23:57:51,0.4215,UnikoinGold the First Crypto Officially Approv...,-0.000,0.833,0.167,1
3,1.525096e+09,2,UnikoinGold the First Crypto Officially Approv...,2018-04-30 08:38:56,0.4215,UnikoinGold the First Crypto Officially Approv...,-0.000,0.833,0.167,1
4,1.525096e+09,2,UnikoinGold the First Crypto Officially Approv...,2018-04-30 08:38:56,0.4215,UnikoinGold the First Crypto Officially Approv...,-0.000,0.833,0.167,1
5,1.525237e+09,1,Buy CBXE Tokens! And become a part of the subs...,2018-05-01 23:52:46,0.2714,Buy CBXE Tokens! And become a part of the subs...,-0.000,0.901,0.099,1
8,1.525233e+09,1,Dubai police arrests gang for stealing Bitcoin...,2018-05-01 22:53:16,-0.7650,Dubai police arrests gang for stealing Bitcoin...,-0.524,0.476,0.000,-1


In [121]:
headline1_df_filter.to_csv('semtiment_score.csv',sep = '\t')

In [ ]:

pos = headline1_df_filter.groupby('timestamp')['pos'].agg(['sum','count'])

In [109]:
neg = headline1_df_filter.groupby('timestamp')['neg'].agg(['sum','count'])

In [119]:
pos[pos['sum'] > 0.5]

,sum,count
timestamp,,
2018-04-19 21:19:58,0.524,1
2018-04-20 04:31:25,0.530,1
2018-04-20 19:01:08,0.600,1
2018-04-21 07:02:39,0.562,1
2018-04-21 11:02:23,0.528,1
2018-04-22 08:42:49,0.583,1
2018-04-24 16:57:58,0.511,1
2018-04-24 21:26:31,0.767,1
2018-04-26 09:14:32,0.529,1
